https://www.youtube.com/watch?v=3G3zExNItj0&t=303&ab_channel=%EC%A1%B0%EC%BD%94%EB%94%A9JoCoding

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
import seaborn as sns
import tqdm

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier            # Voting 기법 사용 Module
from sklearn.neighbors import KNeighborsClassifier 
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

Data Load

In [12]:
df = pd.read_csv('./data/lotto.csv', index_col=0)
df

ValueError: could not convert string 'num,1,2,3,4,5,6,bonus,1st,2nd,3rd,4th,5th' to float64 at row 0, column 1.

In [4]:
df.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086],
           dtype='int64', name='num', length=1086)

In [5]:
df.columns

Index(['1', '2', '3', '4', '5', '6', 'bonus', '1st', '2nd', '3rd', '4th',
       '5th'],
      dtype='object')

One Hot Encoding

In [6]:
# one-hot encoding

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers
     

In [24]:
rows = df.values
row_count = len(rows)

numbers = rows[:, 1:7]
lotto_onehot = list(map(numbers2ohbin, numbers))

x_samples = lotto_onehot[0:row_count-1]
y_samples = lotto_onehot[1:row_count]

#원핫인코딩으로 표시
print("lotto_onehot")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

lotto_onehot
X[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
Y[0]: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
numbers
X[0]: [16, 23, 29, 33, 37, 40]
Y[0]: [2, 13, 21, 25, 32, 42]


In [29]:
lotto_onehot

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 

In [39]:
lotto_df = pd.DataFrame(lotto_onehot, columns=np.arange(1,46), index = np.arange(1, 1087))
lotto_df = lotto_df.astype(int)
lotto_df

,1,2,3,4,5,6,7,8,9,10,...,36,37,38,39,40,41,42,43,44,45
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1083,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1084,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1085,0,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0


Modeling

In [41]:
# train test spilt 

X = lotto_df.index
y = lotto_df.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=777)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(868,) (868, 45)
(218,) (218, 45)


In [46]:
y[0]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [49]:
# Model Load
dt = DecisionTreeClassifier()
# rf = RandomForestClassifier()
# xgb = xgb.XGBClassifier()
# lgb = lgb.LGBMClassifier()

# Training 
dt.fit(X_train, y_train[0].reshape(-1, 1))
# rf.fit(X_train, y_train)
# xgb.fit(X_train, y_train)
# lgb.fit(X_train, y_train)

# Predict 
dt_pred = dt.predict(X_val)
# rf_pred = rf.predict(X_val)
# xgb_pred = xgb.predict(X_val)
# lgb_pred = lgb.predict(X_val)

ValueError: Expected 2D array, got 1D array instead:
array=[5.500e+01 7.510e+02 8.300e+01 9.800e+01 4.870e+02 5.920e+02 7.900e+01
 9.270e+02 4.330e+02 6.300e+02 7.760e+02 1.980e+02 4.000e+01 6.160e+02
 7.600e+01 1.480e+02 1.600e+01 4.610e+02 1.020e+03 8.860e+02 8.020e+02
 1.940e+02 7.970e+02 1.120e+02 8.900e+02 1.520e+02 5.380e+02 1.790e+02
 5.680e+02 4.150e+02 7.220e+02 1.930e+02 6.060e+02 7.730e+02 4.350e+02
 7.630e+02 9.810e+02 9.940e+02 1.083e+03 5.400e+01 9.740e+02 2.370e+02
 3.210e+02 1.330e+02 7.500e+02 1.000e+01 6.900e+01 5.900e+02 6.630e+02
 5.600e+02 2.980e+02 4.950e+02 6.450e+02 1.200e+02 2.900e+02 2.560e+02
 1.008e+03 8.140e+02 5.130e+02 2.810e+02 7.820e+02 1.130e+02 9.460e+02
 2.540e+02 1.920e+02 5.320e+02 1.020e+02 4.740e+02 2.180e+02 2.700e+02
 6.660e+02 3.870e+02 5.910e+02 1.750e+02 8.040e+02 8.450e+02 9.310e+02
 3.160e+02 5.500e+02 4.770e+02 2.250e+02 2.070e+02 9.950e+02 8.750e+02
 4.980e+02 3.600e+01 6.480e+02 1.710e+02 8.270e+02 1.470e+02 3.810e+02
 1.860e+02 8.310e+

Evaluation

In [ ]:
# accuracy, recall, precision, auc graph, confusion matrix

def eval_model(pred_list, name_list, y_test):
    for pred, name in zip(pred_list, name_list):
        accuracy = accuracy_score(y_test , pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        auc = roc_auc_score(y_test, pred)
        matrix = confusion_matrix(y_test, pred)

        plt.figure(figsize=(8,3))
        plt.subplot(1, 2, 1)
        fpr, tpr, thresholds = roc_curve(y_val, pred, pos_label=1)
        # roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, color='darkorange')
        plt.plot([0, 1], [0, 1], color = 'black', label = 'y = x')
        plt.subplot(1, 2, 2)
        sns.heatmap(matrix, annot=True, fmt='g') # fmt='g' : 지수 표기를 일반 표기로
        plt.title('{0} Model Confusion Matrix'.format(name))
        plt.show()
        plt.tight_layout()

        print('{0} 정확도: {1:.4f}'.format(name, accuracy))
        print('{0} Recall: {1:.4f}'.format(name, recall))
        print('{0} Precision: {1:.4f}'.format(name, precision))
        print('{0} AUC: {1:.4f}'.format(name, auc))

        print('='* 50)
        print()